# Midterm. News classification
- Implement news classification using pytorch sentiment analysis.
- Using `BalancedNewsCorpus_train.csv` as train set, `BalancedNewsCorpus_test.csv` as test set. It is a balanced data for 9 newspapers among news data from NIKL.
- 3 Word2Vec models, 4 fasttext models and 1 Glove models was provided for performance comparison.
  - Models were defined by multiple features : space, morph, jamo etc.
  - They were not uploaded due to copyright issues.

In [ ]:
'''
!pip install gensim
!pip install --ignore-installed hanja
!sudo apt-get install python-dev; pip install konlpy
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
'''

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.utils.data
from torchtext import data
from torchtext import datasets
import random
import tqdm
import konlpy
from konlpy.tag import Mecab

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Data loading and preprocessing
- sentence cleansing has not been processed due to Mecab tokenizer.
- `Mecab.nouns` function provide high quality preprocessing for korean sentences.

In [ ]:
train_df = pd.read_csv('BalancedNewsCorpus_train.csv', encoding='utf-8')
test_df = pd.read_csv('BalancedNewsCorpus_test.csv', encoding='utf-8')

train_df.head(5)

In [ ]:
def tokenizer (list):
    mecab = Mecab()
    total_nouns = []
    
    for string in list:
        nouns= mecab.nouns(string) # provide preprocessing
        nouns = [n for n in nouns if len(n) >1]
        
        total_nouns += nouns
        
    return total_nouns

## Build data iterator

In [ ]:
TEXT = data.Field(preprocessing= tokenizer, batch_first = True)
LABEL = data.LabelField(sequential=False)

### Define datafields

In [ ]:
from torchtext.data import TabularDataset

raw_datafields = [("filename", None), # we don't need filename, so we pass in None as the field
                 ("date", None), ("NewsPaper", None),
                 ("Topic", LABEL), ("News", TEXT)]

train_data = data.TabularDataset(
        path='BalancedNewsCorpus_train.csv',
        format='csv',
        skip_header=True,
        fields=raw_datafields)

test_data = data.TabularDataset(
        path='BalancedNewsCorpus_test.csv',
        format='csv',
        skip_header=True,
        fields=raw_datafields)

print(vars(test_data.examples[3]))

### Build vocab 

In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

print(f"The example of text vocab : {TEXT.vocab.itos[:10]}")
print(f"Token index : {LABEL.vocab.stoi}")

### Create dataiterator

In [ ]:
BATCH_SIZE = 64

train_iterator = data.BucketIterator(
    dataset=train_data, batch_size=BATCH_SIZE,
    sort_key=lambda x: data.interleave_keys(len(x.News), len(x.Topic)))

test_iterator = data.BucketIterator(
    dataset=test_data, batch_size=BATCH_SIZE,
    sort_key=lambda x: data.interleave_keys(len(x.News), len(x.Topic)))

## Build RNN model (LSTM)
- Record only the best performance.
- LSTM models had the highest test accuracy than other types of CNN models.

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, 
                           dropout=dropout, batch_first=True)

        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        embedded = self.dropout(self.embedding(text))

        output, (hidden, cell) = self.rnn(embedded)
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        return self.fc(hidden)

### Define model hyper-parameters

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 9
N_LAYERS = 1 
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

# Load pre-trained embedding (fasttext, morph)
- Record only the best performance.

In [ ]:
from gensim.models import KeyedVectors

pre_embedding = KeyedVectors.load_word2vec_format('fasttext_morph_300.model', binary=False, encoding='utf-8')

In [ ]:
from tqdm.notebook import tqdm

w2v = []

for token, idx in tqdm(TEXT.vocab.stoi.items()):
    if token in pre_embedding.wv.vocab.keys():
        w2v.append(pre_embedding[token])
    else:
        w2v.append(0.1*np.random.randn(EMBEDDING_DIM))
        
pretrained_embeddings = torch.FloatTensor(w2v)

### Using pre-trained embeddings for weight initialization

In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Training
- Optimizer : Adam, lr=0.001

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    total = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()

        batch.News = batch.News.cuda()
        batch.Topic = batch.Topic.cuda()
        
        predictions = model(batch.News).squeeze(1)
        
        loss = criterion(predictions, batch.Topic)

        _, predicted = torch.max(predictions.data, 1)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        total += batch.Topic.size(0)
        epoch_acc += (predicted == batch.Topic).sum().item()
        
    return epoch_loss, epoch_acc / total

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    total = 0
    
    model.eval()

    with torch.no_grad():
        for batch in iterator:
            batch.News = batch.News.cuda()
            batch.Topic = batch.Topic.cuda()
            predictions = model(batch.News).squeeze(1)
            
            loss = criterion(predictions, batch.Topic)
            
            _, predicted = torch.max(predictions.data, 1)

            epoch_loss += loss.item()
            total += batch.Topic.size(0)
            epoch_acc += (predicted == batch.Topic).sum().item()
        
    return epoch_loss, epoch_acc / total

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 16

best_test_loss = float('inf')
best_test_acc = 0

train_loss = [0 for i in range(N_EPOCHS)]
train_acc =[0 for i in range(N_EPOCHS)]
test_loss = [0 for i in range(N_EPOCHS)]
test_acc = [0 for i in range(N_EPOCHS)]

i = 0

for epoch in range(N_EPOCHS):

    start_time = time.time()

    
    train_loss[i], train_acc[i] = train(model, train_iterator, optimizer, criterion)
    test_loss[i], test_acc[i]= evaluate(model, test_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if test_loss[i] < best_test_loss:
        best_test_loss = test_loss[i]
    
    if test_acc[i] > best_test_acc:
        best_test_acc = test_acc[i]
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss[i]:.3f} | Train Acc: {train_acc[i]*100:.2f}%')
    print(f'\t Test Loss: {test_loss[i]:.3f} |  Test Acc: {test_acc[i]*100:.2f}%')

    i+=1

## Test
####  News label
    -  IT/과학': 0, '경제': 1, '문화': 2, '미용/건강': 3, '사회': 4, '생활': 5, '스포츠': 6, '연예': 7, '정치': 8

In [ ]:
def predict_news(model, sentence, min_len=5):
    model.eval()

    tokenized = [tok for tok in tokenizer(sentence.split())]
    
    if len(tokenized) < min_len:
      tokenized += ['<pad>'] * (min_len - len(tokenized))

    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).unsqueeze(0).to(device)
    prediction = torch.sigmoid(model(tensor))
    out = torch.mean(prediction, 0)
    news_label = torch.max(out, 0)
    
    return {'Predicted Label': LABEL.vocab.itos[int(news_label.indices)], 'Acc.': news_label.values.item(), 'Sentence': sentence}

In [ ]:
## 아래 문장의 정답은 5-8-1-2-4-6-7-3-0
## 연예/문화, 정치/경제/사회/생활 등 명확히 구별되기 어려운 범주들이 있음...

sentence = []
sentence.append("차진 식감과 부드러운 감촉을 모두 지닌 식빵, 결결이 찢어지는 크루아상, 둥글고 크게 구운 호밀빵 모두 모양새부터 알차고 단단했다. 디저트로 눈을 옮기면 국가 대표팀같이 뭐 하나 빼놓을 수 없는 케이크가 나란히 줄을 서 있었다.")
sentence.append("정부는 2012년 예산의 공고안과 배정계획을 1월3일 국무회의에서 의결하고 연초부터 바로 집행에 들어간다. 세계 경제의 불확실성이 높아지고 경기가 둔화할 가능성이 높은 만큼 조기 집행에 박차를 가할 예정이다")
sentence.append("국세청은 특히 서민생활에 피해를 주면서 폭리를 취하는 매점매석 농수산물 유통업체 등에 대한 추적조사를 강화하고, 지방청에 ‘민생침해 사업자 조사전담팀’을 꾸려 민생침해 탈세자에 대한 엄정 대응에 나설 계획이라고 밝혔다")
sentence.append("26일 제25회 부산국제영화제 갈라 프레젠테이션 부문 초청작 '스파이의 아내' 온라인 기자회견이 진행됐다. 작품을 연출한 구로사와 감독이 참석해 작품에 대한 이야기를 나눴다.")
sentence.append("70대 운전사가 몰던 25인승 어린이 통학버스가 주유소로 돌진해 차량 3대를 들이 받았다. 다행히 통학버스에 운전자 외에는 탑승자가 없어 큰 인명피해는 피했다. 운전자는 차량 결함을 주장하고 있으나, 경찰은 운전자 과실 여부도 조사 중이다.")
sentence.append("토트넘이 손흥민에게 주급 20만 파운드(약 3억원)-5년 재계약을 제안할 준비를 마쳤다.' 25일(한국시각) 영국 대중일간 더선의 헤드라인이다. 조제 무리뉴 토트넘 감독이 번리전을 앞두고 기자회견을 통해 구단에 토트넘에서의 손흥민의 장밋빛 미래를 확신하며 재계약을 요청한 직후 영국 현지 언론에선 손흥민 재계약 보도가 쏟아지고 있다.")
sentence.append("공연은 말 그대로 다채로움 그 자체였다. 발레극인지, 현대무용극인지, 전통극인지, 연극인지, 연극이면 다인극인지 1인극인지 모를 정도로 다양한 장르의 결합이 먼저 눈에 띈다.")
sentence.append("발이 아프면 걷는 자세가 나빠지고 자연스럽게 무릎, 골반, 허리에 이상이 생길 수 있다. 이를 예방하려면 평소 발바닥 근육을 스트레칭하고 강화하는 운동을 지속하는 게 중요하다.")
sentence.append("전기차 화재 사고가 연이어 발생하는 가운데 불타지 않는 SK이노베이션 배터리가 주목받는다. SK이노베이션의 배터리는 글로벌 배터리 업체 중 유일하게 단 한건의 화재도 일어나지 않았다. 이같은 비결이 자동차 안정성을 결정짓는 분리막 내재화에 있다는 평가가 나온다.")

raw_data = {'Labels': ['생활','정치','경제','문화','사회','스포츠','연예','미용/건강','IT/과학']}

result = pd.DataFrame(raw_data)

df = pd.DataFrame()
for s in sentence:
  df = df.append(predict_news(model, s), ignore_index=True)

result = pd.concat([result,df],axis=1, join='inner')

Accuracy = sum((result['Labels'] == result['Predicted Label'])==True)/len(result) * 100

print("Accuracy = ",round(Accuracy,2),"%")

result